In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
print('Total Memory: {}'.format(t / 1e9))

Total Memory: 15.812263936


In [ ]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)

In [ ]:
# %%capture
# !pip install simpletransformers==0.49.3

In [ ]:
# from ClassificationModel import ClassificationModel

In [ ]:
!pip install transformers==2.10.0
!pip install simpletransformers==0.26.0

     |████████████████████████████████| 665kB 17.2MB/s 
     |████████████████████████████████| 3.8MB 58.3MB/s 
     |████████████████████████████████| 1.1MB 60.1MB/s 
     |████████████████████████████████| 890kB 55.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a8018f894f3a2eaa524a20b69f5a2178ddc61d31b9c9e192252858194bf5c0ea
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 12.6MB/s 
     |████████████████████████████████| 317kB 28.3MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=f305bf78b2f7f7080005357d6b0f87b4a32330e96db9f3e5ac1e656dd6077dca
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
le = LabelEncoder()

In [ ]:
#train = pd.read_csv('drive/My Drive/data-class/train.csv', names=['text', 'Code', 'country'], header=0)
train = pd.read_csv('drive/My Drive/data-class/train_combinations.csv', names=['text', 'Code'], header=0)
le.fit(train.Code)
train["labels"] = le.transform(train.Code)
train = train.sample(frac=1, random_state=42).reset_index()
train_df = train[["text", "labels"]]
train_df.head(5)

,text,labels
0,I prezzi sono aumentati dello 03% a febbraio e...,17
1,Zapatero decidiu anunciar no dia 2 que não ser...,14
2,"Não basta, portanto, afirmar o princípio abstr...",33
3,Se puso en marcha la mayor operación de abaste...,24
4,Esses incentivos estimulam os investimentos pr...,16


In [ ]:
val = pd.read_csv('drive/My Drive/data-class/val.csv', names=['text', 'Code', 'country'], header=0)
val["labels"] = le.transform(val.Code)
val_df = val[["text", "labels"]]
val_df.head(5)

,text,labels
0,de tal forma que a descentralização prometida ...,12
1,Garantia de regularização dos trabalhadores ru...,40
2,Il dibattito sulla necessità di ampliare l ' o...,29
3,Los dirigentes socialistas de Andalucía promue...,14
4,Potenziare e sviluppare in maniera strutturale...,24


In [ ]:
test = pd.read_csv('drive/My Drive/data-class/test.csv', names=['text', 'Code', 'country'], header=0)
test["labels"] = le.transform(test.Code)
test_df = test[["text", "labels", "country"]]
test_df.head(5)

,text,labels,country
0,La Dirección General culpa a los conductores d...,24,S
1,A instituição de um Estatuto específico que de...,41,P
2,Tal iniciativa deverá ser concebida em estreit...,30,P
3,Londres dice que es 'ilegal' la captura del ba...,5,S
4,Além de atender os desempregados que estão rec...,24,B


In [ ]:
model = ClassificationModel(
                            'xlmroberta',
                            #'xlm-roberta-base',
                            # 'bert',
                            #'drive/My Drive/roberta-class/',
                            # 'drive/My Drive/roberta-class-trans/',
                            # 'drive/My Drive/roberta-class-comb/',
                            # 'bert-base-multilingual-cased', 
                            # 'drive/My Drive/bert-lm/',
                            # 'drive/My Drive/roberta-lm/',
                            # 'drive/My Drive/roberta-lm-comb/',
                            'drive/My Drive/roberta-smooth-0.9/',
                            #'drive/My Drive/roberta-trans-smooth-0.9/',
                            # 'outputs/best_model/',
                            num_labels=len(train_df.labels.unique()), 
                            args={'train_batch_size': 32, 
                                  'eval_batch_size': 32,
                                  'num_train_epochs': 2,
                                  'max_seq_length': 200,
                                  'weight_decay': 0.01,
                                  'adam_epsilon': 1e-6,
                                  'warmup_ratio': 0.1,
                                  'evaluate_during_training': True,
                                  'evaluate_during_training_verbose': True,
                                  'evaluate_during_training_steps': 0,
                                  'save_model_every_epoch': False,
                                  'save_eval_checkpoints': False,
                                  'save_steps': 0,
                                  'use_early_stopping': True,
                                  'early_stopping_patience': 2,
                                  'fp16': False,
                                  'overwrite_output_dir': True,
                                  'reprocess_input_data': True,
                                  'learning_rate': 2e-5,
                                  'manual_seed':42
                                  })

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:173: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


In [ ]:
# model.train_model(train_df, eval_df=val_df)

In [ ]:
from sklearn import metrics

In [ ]:
def f_macro(true, preds):
  return metrics.f1_score(true, preds, average='macro')

def f_micro(true, preds):
  return metrics.f1_score(true, preds, average='micro')

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=metrics.accuracy_score, f1M=f_macro, f1m=f_micro)

In [ ]:
result

{'acc': 0.6201802040545912,
 'eval_loss': 1.4946890251111176,
 'f1M': 0.4376448813153832,
 'f1m': 0.6201802040545912,
 'mcc': 0.5968839250134947}

In [ ]:
import numpy as np

In [ ]:
def softmax(logits):
  return np.exp(logits) / np.sum(np.exp(logits), axis=0)

In [ ]:
preds = [np.argmax(softmax(logits)) for logits in model_outputs]

In [ ]:
label_to_code = dict(zip(train.labels, train.Code))

In [ ]:
test_df["preds"] = preds

In [ ]:
test_df["country"] = test.country

In [ ]:
domain_to_label = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7:[], 9:[]}
for label, code in label_to_code.items():
  domain = code // 100
  domain_to_label[domain].append(label)

In [ ]:
preds_domain = []
for label in preds:
  for domain, labels in domain_to_label.items():
    if label in labels:
      preds_domain.append(domain)

In [ ]:
true_domain = []
for label in test_df.labels:
  for domain, labels in domain_to_label.items():
    if label in labels:
      true_domain.append(domain)

In [ ]:
test_df["true_domain"] = true_domain
test_df["pred_domain"] = preds_domain

In [ ]:
def country_f_scores(df):
  return f_macro(df.labels, df.preds), f_macro(df.true_domain, df.pred_domain)

In [ ]:
{c: country_f_scores(test_df[test_df.country == c]) for c in test_df.country.unique()}

{'B': (0.4161700446205665, 0.6220224787624171),
 'I': (0.2662425794605715, 0.502791445940395),
 'P': (0.31232753977116157, 0.507481468431823),
 'S': (0.5403683318584003, 0.6359581626327986)}

In [ ]:
print("Domain Accuracy: {}".format(metrics.accuracy_score(true_domain, preds_domain)))
print("Domain F-Macro: {}".format(f_macro(true_domain, preds_domain)))
print("Domain F-Micro: {}".format(f_micro(true_domain, preds_domain)))
print("Domain MCC: {}".format(metrics.matthews_corrcoef(true_domain, preds_domain)))

Domain Accuracy: 0.713793560355108
Domain F-Macro: 0.6587707286879891
Domain F-Micro: 0.713793560355108
Domain MCC: 0.6519997825331826


In [ ]:
test_df.to_csv('results_final.csv', index=False)